In [2]:
%run ../explications.ipynb

# Le problème du marathon 

Les six premiers à l'arrivée du marathon de Paris sont
Dominique, Ignace, Naren, Olivier, Philippe et Pascal.
Il s'agit de reconstruire leur ordre  d'arrivée à partir des
informations suivantes :


In [3]:
regles = {
    1 :  "Olivier n'est pas arrivé le dernier",
    2 :  "Dominique, Pascal et Ignace sont arrivés avant Naren et Olivier",
    3 :  "Dominique qui était 3-ème l'année précédente s'est amélioré cette année",
    4 :  "Philippe est parmi le quatre premiers",
    5 :  "Ignace n'est arrivé ni second, ni troisième",
    6 :  "Pascal a battu Naren de trois positions",
    7 :  "Ni Ignace ni Dominique ne sont en quatrième position"
}

## Modélisation

In [5]:
nom = np.array(['Dominique', 'Ignace', 'Naren', 'Olivier', 'Philippe', 'Pascal'], dtype = '<U10')

def model_marathon():
    """
    retourne un quadruplet (vars, facts, constraints, d) tel que
    (vars, facts, constraints) est utilisable dans EXPLANATION
    et d est un dictionnaire : contrainte -> numéro de la contrainte
    """    

    # Model Variables
    place = intvar(1, 6, shape=nom.shape, name = 'place')

    # Pour faciliter l'expression des contraintes :
    for i in range(6): 
        globals()[nom[i]] = place[i]
    
    # initialement, on ne connait la place de personne
    facts = []

    # bijectivity constraint
    cons_b = [AllDifferent(place)]
    
    # rules constraints
    d = {
        Olivier != 6 :            1, 
        (Dominique < Naren) & \
        (Dominique < Olivier) & \
        (Pascal < Naren) & \
        (Pascal < Olivier) & \
        (Ignace < Naren) & \
        (Ignace < Olivier) :      2,
        Dominique < 3 :           3,
        Philippe <= 4 :           4,
        (Ignace != 2) & \
        (Ignace != 3) :           5,
        Pascal == Naren - 3 :     6,
        (Ignace != 4) & \
        (Dominique != 4) :        7
    }
    cons_r = list(d.keys())
    
    return place, facts, cons_b + cons_r, d

## Solution et préparation de l'affichage des explications

In [8]:
vars, facts, constraints, _ = model_marathon()
explanation_marathon = EXPLANATION(vars, facts, constraints)
sol = explanation_marathon.solve()

from termcolor import colored

l = 13

def print_sol(val, facts = set()):
    fmt = '{:>13}' * 6
    print(fmt.format(1,2,3,4,5,6))
    sol = np.zeros(shape=nom.shape, dtype = '<U10')
    m = np.zeros(shape=nom.shape, dtype = 'bool')
    v = np.zeros(shape=nom.shape, dtype = 'bool')
    for fact in facts:
        i,_ = extractFact(fact)
        m[i] = True
    for i in range(6):
        k = val[i] - 1
        if val[i]:
            sol[k] = nom[i]
            v[k] = m[i]    
    for k in range(6):
            s = sol[k]
            if v[k]:
                print(' ' * (l - len(s)) + colored(s.upper(),'blue', attrs =['bold'],force_color=True), end = '')
            else:
                print(' ' * (l - len(s)) + s, end = '')
    print('')

print_sol(sol)

            1            2            3            4            5            6
       Ignace    Dominique       Pascal     Philippe      Olivier        Naren


## Explications pour la résolution complète

In [7]:
d = model_marathon()[3]
ESNlist = explanation_marathon.explainFull(verbose = False)

In [9]:
val = np.zeros(shape = nom.shape, dtype = 'int32')

def add_facts(facts):
    for fact in facts:
        i, v = extractFact(fact)
        val[i] = v
    
for E,S,N in ESNlist:
    constraints = set()
    print_sol(val, facts = E) 
    print('')
    for r in S:
        if r.name != 'alldifferent':
            print(colored(regles[d[r]], 'blue', attrs =['bold'], force_color=True))
    add_facts(N)
    i, v = extractHint(N)
    print('\ndonc ' + colored(f'{nom[i]} est arrivé {v}-ème\n', 'green', force_color=True) + '_' * 78 + '\n')
    
print_sol(val)



            1            2            3            4            5            6
                                                                              

Ignace n'est arrivé ni second, ni troisième
Ni Ignace ni Dominique ne sont en quatrième position
Dominique, Pascal et Ignace sont arrivés avant Naren et Olivier

donc Ignace est arrivé 1-ème
______________________________________________________________________________

            1            2            3            4            5            6
       IGNACE                                                                 

Dominique qui était 3-ème l'année précédente s'est amélioré cette année

donc Dominique est arrivé 2-ème
______________________________________________________________________________

            1            2            3            4            5            6
       IGNACE    Dominique                                                    

Dominique qui était 3-ème l'année précédente s'est amélioré cette a